In [1]:
## spark version
import os
import sys

spark_home = os.environ['SPARK_HOME']
sys.path.insert(0, spark_home + "/python/")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0-SNAPSHOT
      /_/

Using Python version 2.7.10 (default, May 28 2015 17:02:03)
SparkContext available as sc, SQLContext available as sqlContext.


In [100]:
articles = sc.newAPIHadoopFile("Reuters/lyrl2004_tokens_*.dat", 
                    "org.apache.hadoop.mapreduce.lib.input.TextInputFormat",
                    "org.apache.hadoop.io.LongWritable", 
                    "org.apache.hadoop.io.Text",
                    conf={"textinputformat.record.delimiter": ".I "}) \
    .map(lambda l:l[1]) \
    .filter(lambda x: x != u'')


In [106]:
articles.take(2)

[u'26151\n.W\nsocc colomb colomb colomb colomb beat beat chil chil chil chil world world world cup cup cup qualif qualif qualif qualif halftim south south americ americ match sunday scor faustin\nasprill st minut minut jorg bermudez ivan zamoran penalt attend group stand tabulat play won draw lost goal goal point ecuador argentin boliv paraguay uruguay peru venezuel note top final franc\nbrazil automat hold\n\n',
 u'26152\n.W\nworld world world world qualif qualif sunday minut minut won hold athlet time time time time time lucky lucky komen komen komen komen break break record record record daniel keny\nkeny keny keny keny keny made shat noureddin morcel morcel morcel morcel morcel morcel morcel met met met met met met second second second second intern meet year year fail fail\natlant olymp olymp olymp olymp clock clock clock clock set set ago mont carl blist form grand grand prix prix circuit mark mark monac month brussel august fast fast hist hist\nfinish finish back back sixth plac

In [107]:
import re
idspot = re.compile(r"^(\d*)\n.W\n")

def wrdcount(x):
    try:
        did = idspot.match(x).group(1) 
        wrds = re.sub('\n',' ', idspot.sub('',x)).split()
        counter = wrds.count 
        return [(w,u"{0}|{1}".format(did,counter(w))) for w in set(wrds)]
    except:
        return None


In [108]:
tokens = articles.flatMap(wrdcount).sortByKey( )

In [110]:
tokens.take(10)

[(u'_', u'38911|2'),
 (u'_', u'39049|2'),
 (u'_', u'58914|1'),
 (u'_', u'67020|1'),
 (u'_', u'72416|1'),
 (u'_', u'80545|1'),
 (u'_', u'97538|1'),
 (u'_', u'118505|1'),
 (u'_', u'122529|2'),
 (u'_', u'122697|8')]

In [111]:
lines = tokens.map( lambda x: u"{0}|{1}".format(x[0],x[1]) )
lines.saveAsTextFile('reutersTokens')